In [14]:
# Import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import LinearSVC
from scipy.stats import randint
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,roc_auc_score,classification_report,roc_auc_score,roc_curve,auc
from sklearn.cross_validation import *
import xgboost as xgb

# Figures inline and set visualization style
%matplotlib inline
sns.set()

In [4]:
# import data
data_train = pd.read_csv(r'C:\Installs\Data science\blood_donation_prediction\train.csv')
data_test = pd.read_csv(r'C:\Installs\Data science\blood_donation_prediction\test.csv')


In [5]:
data_test.head()


,Unnamed: 0,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation
0,659,2,12,3000,52
1,276,21,7,1750,38
2,263,4,1,250,4
3,303,11,11,2750,38
4,83,4,12,3000,34


In [6]:
#sns.countplot(x='Made Donation in March 2007', data=data_train);

In [7]:
#sns.stripplot(x='Made Donation in March 2007',y='Months since Last Donation', data=data_train)

In [8]:
#sns.stripplot(x='Made Donation in March 2007',y='Months since First Donation', data=data_train)

In [9]:
#sns.scatterplot(x='Made Donation in March 2007',y='Number of Donations', data=data_train)

In [10]:
#bulding model: transform dataframe to array
X = data_train.drop(['Unnamed: 0','Made Donation in March 2007'],axis=1).values
test = data_test.values
y = data_train['Made Donation in March 2007'].values

In [11]:
logreg = LogisticRegression()
logreg.fit(X,y)
logreg.score(X,y)

0.7638888888888888

In [12]:
gbc = GradientBoostingClassifier()
gbc.fit(X,y)
gbc.score(X,y)

0.8611111111111112

In [19]:
xgb = xgb.XGBClassifier()

xgb.fit(X,y)
xgb.score(X,y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8402777777777778

In [20]:
gb_grid_params = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[20]
}

gb_gs = GradientBoostingClassifier()

clf = GridSearchCV(gb_gs,
                               gb_grid_params,
                               cv=2,
                               scoring='roc_auc',
                               verbose = 3, 
                               n_jobs=10);
clf.fit(X, y);

# Print the tuned parameter and score
print("Tuned Decision Tree Parameters: {}".format(clf.best_params_))
print("Best score is {}".format(clf.best_score_))



Fitting 2 folds for each of 84672 candidates, totalling 169344 fits


[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    3.6s
[Parallel(n_jobs=10)]: Done 467 tasks      | elapsed:    4.9s
[Parallel(n_jobs=10)]: Done 2387 tasks      | elapsed:    9.1s
[Parallel(n_jobs=10)]: Done 5075 tasks      | elapsed:   15.6s
[Parallel(n_jobs=10)]: Done 8531 tasks      | elapsed:   24.6s
[Parallel(n_jobs=10)]: Done 12755 tasks      | elapsed:   34.3s
[Parallel(n_jobs=10)]: Done 17747 tasks      | elapsed:   45.2s
[Parallel(n_jobs=10)]: Done 23507 tasks      | elapsed:   59.7s
[Parallel(n_jobs=10)]: Done 30035 tasks      | elapsed:  1.3min
[Parallel(n_jobs=10)]: Done 37331 tasks      | elapsed:  1.5min
[Parallel(n_jobs=10)]: Done 45395 tasks      | elapsed:  1.8min
[Parallel(n_jobs=10)]: Done 54227 tasks      | elapsed:  2.2min
[Parallel(n_jobs=10)]: Done 63827 tasks      | elapsed:  2.6min
[Parallel(n_jobs=10)]: Done 74195 tasks      | elapsed:  3.0min
[Parallel(n_jobs=10)]: Done 85331 tasks      | elapsed:  3.4min
[Parallel(n_jobs=10)]: Done 97235 tasks      | 

Tuned Decision Tree Parameters: {'criterion': 'friedman_mse', 'learning_rate': 0.15, 'loss': 'deviance', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 0.13636363636363638, 'min_samples_split': 0.13636363636363638, 'n_estimators': 20, 'subsample': 0.5}
Best score is 0.7717722189133744


In [17]:
xgb_model = xgb.XGBClassifier()


parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['binary:logistic'],
              'learning_rate': [0.05], #so called `eta` value
              'max_depth': [6],
              'min_child_weight': [11],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [1000], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [1337]}


clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=StratifiedKFold(y, n_folds=5, shuffle=True), 
                   scoring='roc_auc',
                   verbose=2, refit=True)

clf.fit(X, y)
# Print the tuned parameter and score
print("Tuned Decision Tree Parameters: {}".format(clf.best_params_))
print("Best score is {}".format(clf.best_score_))

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    3.0s remaining:    4.6s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    4.8s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    4.8s finished


Tuned Decision Tree Parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 11, 'missing': -999, 'n_estimators': 1000, 'nthread': 4, 'objective': 'binary:logistic', 'seed': 1337, 'silent': 1, 'subsample': 0.8}
Best score is 0.7217882628338207


In [13]:
# Make predictions and store in 'Survived' column of df_test
Y_pred = gbc.predict_proba(test)
data_test['Made Donation in March 2007'] = Y_pred[:,1]


In [15]:
data_test[['Unnamed: 0', 'Made Donation in March 2007']].to_csv('kotharv_blooddon_gb_tree.csv', index=False)